In [ ]:
import tensorflow as tf
import numpy as np
import scipy
import os
from tensorflow_model_optimization.python.core.keras.compat import keras
import time 

print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"SciPy version: {scipy.__version__}")



In [ ]:
if tf.config.list_physical_devices('GPU'):
    print("GPU está conectada")
    gpus = tf.config.list_physical_devices('GPU')
    for gpu in gpus:
        print(f"Nome da GPU: {gpu.name}")
else:
    print("Não conectado a uma GPU")

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, average_precision_score
from scipy.stats import ks_2samp
#import scikitplot as skplt
import matplotlib.pyplot as plt
from numpy import interp

from scipy.stats import ks_2samp
from sklearn.metrics import roc_curve, auc


def extract_final_losses(history):
    """Função para extrair o melhor loss de treino e validação.

    Argumento(s):
    history -- Objeto retornado pela função fit do keras.

    Retorno:
    Dicionário contendo o melhor loss de treino e de validação baseado
    no menor loss de validação.
    """
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    idx_min_val_loss = np.argmin(val_loss)
    return {'train_loss': train_loss[idx_min_val_loss], 'val_loss': val_loss[idx_min_val_loss]}

def plot_training_error_curves(history):
    """Função para plotar as curvas de erro do treinamento da rede neural.

    Argumento(s):
    history -- Objeto retornado pela função fit do keras.

    Retorno:
    A função gera o gráfico do treino da rede e retorna None.
    """
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']

    fig, ax = plt.subplots()
    ax.plot(train_loss, label='Train')
    ax.plot(val_loss, label='Validation')
    ax.set(title='Training and Validation Error Curves', xlabel='Epochs', ylabel='Loss (MSE)')
    ax.legend()
    plt.show()

def compute_performance_metrics(y, y_pred_class, y_pred_scores=None):
    accuracy = accuracy_score(y, y_pred_class)
    recall = recall_score(y, y_pred_class)
    precision = precision_score(y, y_pred_class)
    f1 = f1_score(y, y_pred_class)
    performance_metrics = (accuracy, recall, precision, f1)
    if y_pred_scores is not None:
        # skplt.metrics.plot_ks_statistic(y, y_pred_scores)
        # plt.savefig("ks_plot.png")
        # plt.show()
        y_pred_scores = y_pred_scores[:, 1]
        auroc = roc_auc_score(y, y_pred_scores)
        aupr = average_precision_score(y, y_pred_scores)
        performance_metrics = performance_metrics + (auroc, aupr)
    return performance_metrics

def print_metrics_summary(accuracy, recall, precision, f1, auroc=None, aupr=None):
    print()
    print("{metric:<18}{value:.4f}".format(metric="Accuracy:", value=accuracy))
    print("{metric:<18}{value:.4f}".format(metric="Recall:", value=recall))
    print("{metric:<18}{value:.4f}".format(metric="Precision:", value=precision))
    print("{metric:<18}{value:.4f}".format(metric="F1:", value=f1))
    if auroc is not None:
        print("{metric:<18}{value:.4f}".format(metric="AUROC:", value=auroc))
    if aupr is not None:
        print("{metric:<18}{value:.4f}".format(metric="AUPR:", value=aupr))

In [ ]:
path = "/home/pedro/projetoDL/dataset/processado/"
save_path = "/home/pedro/projetoDL/log/prunning/fold3/"

# Verifica se o diretório existe, se não, cria o diretório
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Carregar os dados
Y = np.load(path + 'Y_train_NewApproach_Injected_v2.npz')
Y= Y.f.arr_0

X = np.load(path + 'X_train_NewApproach_Injected_v2.npz')
X = X.f.arr_0

In [ ]:
import tensorflow_model_optimization as tfmot
from sklearn.model_selection import KFold
import numpy as np
import os

# Configurar o KFold e selecionar o fold 3
skf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 0

for train_index, val_index in skf.split(X, Y):
    fold_no += 1
    if fold_no == 3:  # Selecionar o fold 3
        x_train, x_val = X[train_index], X[val_index]
        y_train, y_val = Y[train_index], Y[val_index]
        break



# Verificar os dados
print(f"Tamanho de x_train: {x_train.shape}, y_train: {y_train.shape}")
print(f"Tamanho de x_val: {x_val.shape}, y_val: {y_val.shape}")
print("Distribuição em y_train:", np.unique(y_train, return_counts=True))
print("Distribuição em y_val:", np.unique(y_val, return_counts=True))

model for prunning

In [ ]:
from tensorflow_model_optimization.python.core.keras.compat import keras

file_name = f'log_modelo_fold3.txt'

model_prunned_save_path = "/home/pedro/projetoDL/log/prunning/fold3/prunned/"

file1 = open(os.path.join(model_prunned_save_path, file_name), "a")

model = keras.models.load_model(model_prunned_save_path + '3xPrunned.h5')

start = time.time()
y_pred_scores = model.predict(x_val)
end = time.time()

print(f"\nEvaluate 3xPrunned at GPU\n", file=file1)
print(f"Runtime of the program is {end - start}", file=file1)
total_time = end - start
timesample = total_time/y_pred_scores.shape[0]
print(f"us/sample is {timesample*1000000}", file=file1)

y_pred_class = (y_pred_scores > 0.5).astype("int64")

y_pred_scores2 = y_pred_scores
y_pred_scores_0 = 1 - y_pred_scores
y_pred_scores = np.concatenate([y_pred_scores_0, y_pred_scores], axis=1)

accuracy, recall, precision, f1, auroc, aupr = compute_performance_metrics(y_val, y_pred_class, y_pred_scores)


file1.close()

Criando Poda Extrema

In [ ]:
from tensorflow.keras.models import load_model

# Caminho do modelo a ser carregado
model_path = "/home/pedro/projetoDL/log/prunning/fold3/full/fold3_only.h5"

# Carregar o modelo na variável `model`
model = keras.models.load_model(model_path)

# Verificar o resumo do modelo para garantir que foi carregado corretamente
model.summary()

In [ ]:
import tensorflow_model_optimization as tfmot
import tensorflow as tf
import numpy as np

# Poda extrema (99%)
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.80,
        final_sparsity=0.99,
        begin_step=0,
        end_step=100
    )
}

# Envolver o modelo com poda
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
model_for_pruning = prune_low_magnitude(model, **pruning_params)

# Compilar o modelo
model_for_pruning.compile(
    optimizer=keras.optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Treinar o modelo com callbacks de poda
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
]

model_for_pruning.fit(
    x_train, y_train,
    batch_size=256,
    epochs=5,
    validation_split=0.1,
    callbacks=callbacks
)

# Remover wrappers de poda para exportação
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

# Salvar como H5 para GPU
model_for_export.save("model_pruned_gpu.h5")


In [ ]:
# Converter para TFLite Quantizado para Raspberry Pi
def representative_data_gen():
    for input_value in x_val[:100]:  # Use algumas amostras
        yield [np.expand_dims(input_value, axis=0).astype(np.float32)]

converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

quantized_model = converter.convert()

# Salvar o modelo TFLite
with open("model_pruned_quantized_rpi.tflite", "wb") as f:
    f.write(quantized_model)

avaliando fully prunned

In [9]:
from tensorflow_model_optimization.python.core.keras.compat import keras

file_name = f'log_modelo_full_prunned.txt'

model_prunned_save_path = "/home/pedro/projetoDL/log/prunning/fold3/FULL-PRUNNED/"

file1 = open(os.path.join(model_prunned_save_path, file_name), "a")

model = keras.models.load_model(model_prunned_save_path + 'model_pruned_gpu.h5')

start = time.time()
y_pred_scores = model.predict(x_val)
end = time.time()

print(f"\nEvaluate full prunned at GPU\n", file=file1)
print(f"Runtime of the program is {end - start}", file=file1)
total_time = end - start
timesample = total_time/y_pred_scores.shape[0]
print(f"us/sample is {timesample*1000000}", file=file1)

y_pred_class = (y_pred_scores > 0.5).astype("int64")

y_pred_scores2 = y_pred_scores
y_pred_scores_0 = 1 - y_pred_scores
y_pred_scores = np.concatenate([y_pred_scores_0, y_pred_scores], axis=1)

accuracy, recall, precision, f1, auroc, aupr = compute_performance_metrics(y_val, y_pred_class, y_pred_scores)


file1.close()

  61/4021 [..............................] - ETA: 6s

W0000 00:00:1732500346.486925  205171 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732500346.488148  205171 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732500346.488935  205171 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732500346.490700  205171 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732500346.492047  205171 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732500346.493099  205171 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732500346.493939  205171 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732500346.496166  205171 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732500346.497216  205171 gp

4021/4021 [==============================] - 7s 2ms/step


W0000 00:00:1732500352.978079  205157 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732500352.979174  205157 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732500352.981389  205157 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732500352.982331  205157 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732500352.983263  205157 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732500352.984205  205157 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732500352.984981  205157 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732500352.986686  205157 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732500352.987625  205157 gp